In [1]:
import os
os.add_dll_directory("D:/OpenSim 4.4/bin")  # Add the OpenSim DLL directory

import opensim as osim
import numpy as np
import pandas as pd
import time
import math

# Main Code

## Test the opensim with GPT

In [59]:
import os
os.add_dll_directory("D:/OpenSim 4.4/bin")  # Add the OpenSim DLL directory

import opensim as osim
import numpy as np
import time
import math

# --------------------------------------------------------------------
# 1. Load the Rajagopal 2015 OpenSim model and enable the visualizer.
# --------------------------------------------------------------------
model = osim.Model('Rajagopal_2015.osim')
model.setUseVisualizer(True)
state = model.initSystem()
visualizer = model.getVisualizer()

# --------------------------------------------------------------------
# 2. Define IMU placements on model bodies.
#    Adjust these body names based on your model's actual segments.
# --------------------------------------------------------------------
imu_bodies = ['pelvis', 'femur_r', 'tibia_r']

# --------------------------------------------------------------------
# 3. Define a mapping from body to coordinate and the corresponding Euler angle.
#    For demonstration, we assume:
#      - 'pelvis': update "pelvis_tilt" with the pitch angle
#      - 'femur_r': update "hip_flexion_r" with the roll angle
#      - 'tibia_r': update "knee_angle_r" with the yaw angle
#    Adjust the mapping as necessary.
# --------------------------------------------------------------------
coordinate_mapping = {
    'pelvis':      ('pelvis_tilt',    'pitch'),
    'femur_r':     ('hip_flexion_r',  'roll'),
    'tibia_r':     ('knee_angle_r',   'yaw')
}

# --------------------------------------------------------------------
# 4. Simulate reading IMU data (replace this with your actual sensor code).
# --------------------------------------------------------------------
def read_imu_data():
    imu_data = {}
    for body in imu_bodies:
        # Generate a random quaternion [w, x, y, z] and normalize it.
        quat = np.random.rand(4)
        quat = quat / np.linalg.norm(quat)
        imu_data[body] = quat
    return imu_data

# --------------------------------------------------------------------
# 5. Function to convert quaternion to Euler angles (roll, pitch, yaw).
#    Assumes quaternion is in [w, x, y, z] format.
# --------------------------------------------------------------------
def quat_to_euler(quat):
    w, x, y, z = quat

    # Roll (x-axis rotation)
    t0 = 2.0 * (w * x + y * z)
    t1 = 1.0 - 2.0 * (x * x + y * y)
    roll = math.atan2(t0, t1)

    # Pitch (y-axis rotation)
    t2 = 2.0 * (w * y - z * x)
    t2 = max(min(t2, 1.0), -1.0)  # Clamp to [-1,1]
    pitch = math.asin(t2)

    # Yaw (z-axis rotation)
    t3 = 2.0 * (w * z + x * y)
    t4 = 1.0 - 2.0 * (y * y + z * z)
    yaw = math.atan2(t3, t4)

    return roll, pitch, yaw

# --------------------------------------------------------------------
# 6. Update the model's state using IMU data by converting quaternions to Euler angles
#    and then mapping them to joint coordinates.
# --------------------------------------------------------------------
def update_state_from_imus(state, imu_data):
    for body, quat in imu_data.items():
        roll, pitch, yaw = quat_to_euler(quat)
        print(f"IMU data for {body}: roll={roll:.3f}, pitch={pitch:.3f}, yaw={yaw:.3f}")
        
        # Check if this body has a mapping to a model coordinate.
        if body in coordinate_mapping:
            coord_name, angle_component = coordinate_mapping[body]
            # Choose the corresponding Euler angle.
            if angle_component == 'roll':
                value = roll
            elif angle_component == 'pitch':
                value = pitch
            elif angle_component == 'yaw':
                value = yaw
            else:
                continue  # If mapping is not recognized, skip update.
            
            # Retrieve and update the coordinate value.
            coord = model.getCoordinateSet().get(coord_name)
            coord.setValue(state, value)
            print(f"Updated coordinate '{coord_name}' to {value:.3f} (from {angle_component}).")
        else:
            print(f"No coordinate mapping defined for {body}.")
    return state

# --------------------------------------------------------------------
# 7. Real-time simulation loop: update kinematics and refresh the 3D view.
# --------------------------------------------------------------------
try:
    t = state.getTime()
    dt = 0.1         # Time step in seconds
    final_time = 10.0  # Total simulation duration in seconds
    
    while t < final_time:
        # Acquire simulated IMU data.
        imu_data = read_imu_data()
        
        # Update model state based on IMU data.
        state = update_state_from_imus(state, imu_data)
        
        # Advance simulation time.
        t += dt
        state.setTime(t)
        
        # Recompute kinematics.
        model.realizePosition(state)
        
        # Refresh the visualizer.
        visualizer.show(state)
        
        # Pause to simulate real-time update.
        time.sleep(dt)
except KeyboardInterrupt:
    print("Real-time visualization terminated.")


IMU data for pelvis: roll=0.656, pitch=-0.171, yaw=0.734
Updated coordinate 'pelvis_tilt' to -0.171 (from pitch).
IMU data for femur_r: roll=1.700, pitch=-0.672, yaw=1.150
Updated coordinate 'hip_flexion_r' to 1.700 (from roll).
IMU data for tibia_r: roll=1.441, pitch=0.153, yaw=1.824
Updated coordinate 'knee_angle_r' to 1.824 (from yaw).
IMU data for pelvis: roll=1.358, pitch=0.071, yaw=1.362
Updated coordinate 'pelvis_tilt' to 0.071 (from pitch).
IMU data for femur_r: roll=0.653, pitch=0.222, yaw=0.942
Updated coordinate 'hip_flexion_r' to 0.653 (from roll).
IMU data for tibia_r: roll=2.102, pitch=-0.424, yaw=0.602
Updated coordinate 'knee_angle_r' to 0.602 (from yaw).
IMU data for pelvis: roll=1.280, pitch=0.395, yaw=1.502
Updated coordinate 'pelvis_tilt' to 0.395 (from pitch).
IMU data for femur_r: roll=0.505, pitch=0.012, yaw=1.461
Updated coordinate 'hip_flexion_r' to 0.505 (from roll).
IMU data for tibia_r: roll=1.412, pitch=-0.784, yaw=0.997
Updated coordinate 'knee_angle_r' to

RuntimeError: std::exception in 'void OpenSim::ModelVisualizer::show(SimTK::State const &) const': SimTK Exception thrown at VisualizerProtocol.cpp:456:
  Error detected by Simbody method VisualizerProtocol: An attempt to write() 1 bytes to pipe 18 failed with errno=22 (Invalid argument).
  (Required condition 'status!=-1' was not met.)


In [62]:
import os
os.add_dll_directory("D:/OpenSim 4.4/bin")  # Add OpenSim DLL directory

import opensim as osim
import numpy as np
import time
import math

# --------------------------------------------------------------------
# 1. Load the Rajagopal 2015 model and enable visualization.
# --------------------------------------------------------------------
model = osim.Model('Rajagopal_2015.osim')
model.setUseVisualizer(True)
state = model.initSystem()
visualizer = model.getVisualizer()

# --------------------------------------------------------------------
# 2. Define the IMU placements.
# --------------------------------------------------------------------
imu_bodies = ['pelvis', 'femur_r', 'tibia_r']

# --------------------------------------------------------------------
# 3. Simulate reading IMU data.
#    (Replace this with your actual sensor interfacing code.)
#    Each IMU produces a quaternion [w, x, y, z].
# --------------------------------------------------------------------
def read_imu_data():
    imu_data = {}
    for body in imu_bodies:
        quat = np.random.rand(4)
        quat = quat / np.linalg.norm(quat)
        imu_data[body] = quat
    return imu_data

# --------------------------------------------------------------------
# 4. Convert a quaternion to an OpenSim Rotation.
# --------------------------------------------------------------------
def quat_to_rotation(quat):
    w, x, y, z = quat
    # Compute the rotation matrix corresponding to the quaternion.
    R = np.array([
        [1 - 2 * y * y - 2 * z * z, 2 * x * y - 2 * w * z,     2 * x * z + 2 * w * y],
        [2 * x * y + 2 * w * z,     1 - 2 * x * x - 2 * z * z, 2 * y * z - 2 * w * x],
        [2 * x * z - 2 * w * y,     2 * y * z + 2 * w * x,     1 - 2 * x * x - 2 * y * y]
    ])
    # Create an OpenSim Rotation from the matrix.
    return osim.Rotation(R[0,0], R[0,1], R[0,2],
                           R[1,0], R[1,1], R[1,2],
                           R[2,0], R[2,1], R[2,2])

# --------------------------------------------------------------------
# 5. Set up inverse kinematics using orientation tracking (OpenSense approach).
#
#    The idea is to create an IK tool that uses orientation tracking goals,
#    where each goal is associated with an IMU sensor (i.e. a model body).
#
#    Note: The following classes/methods (e.g. OpenSenseTool, OrientationTrackingGoal)
#    are part of the OpenSense toolkit (or represent a custom implementation).
# --------------------------------------------------------------------
# Create an instance of the OpenSenseTool, which wraps the inverse kinematics process.
openses_tool = osim.OpenSenseTool(model)

# For each IMU, create and add an orientation tracking goal.
# The weight (here 1.0) can be tuned according to your calibration.
for body in imu_bodies:
    # Create an orientation tracking goal for the given body.
    goal = osim.OrientationTrackingGoal(body, 1.0)
    # Add the goal to the OpenSense tool.
    openses_tool.addTrackingGoal(goal)

# --------------------------------------------------------------------
# 6. Real-time loop: update IMU measurements and run IK.
# --------------------------------------------------------------------
try:
    t = state.getTime()
    dt = 0.1         # Time step (seconds)
    final_time = 10.0  # Total simulation duration (seconds)
    
    while t < final_time:
        # Acquire current IMU data (from hardware or simulation).
        imu_data = read_imu_data()
        
        # Update each orientation tracking goal with the measured orientation.
        for body in imu_bodies:
            # Convert IMU quaternion to an OpenSim Rotation.
            measured_rot = quat_to_rotation(imu_data[body])
            # Update the corresponding goal in the OpenSense tool.
            # (Assumes getTrackingGoal(body) retrieves the goal for that sensor.)
            goal = openses_tool.getTrackingGoal(body)
            if goal:
                goal.setDesiredOrientation(measured_rot)
            else:
                print(f"Warning: No tracking goal found for {body}")
        
        # Run the inverse kinematics process to compute joint angles that best
        # match the desired sensor orientations. The IK solution updates the state.
        state = openses_tool.run(state)
        
        # Advance simulation time.
        t += dt
        state.setTime(t)
        
        # Update the model kinematics.
        model.realizePosition(state)
        
        # Refresh the 3D visualizer.
        visualizer.show(state)
        
        # Sleep to mimic real-time updates.
        time.sleep(dt)
except KeyboardInterrupt:
    print("Real-time inverse kinematics visualization terminated.")


AttributeError: module 'opensim' has no attribute 'OpenSenseTool'

## Write my own  Code (random quat)
using the main.py code

### Mine

In [2]:
bodies = ['pelvis_imu', "hand_r_imu", "humerus_r_imu", "radius_r_imu"]

quats = np.random.rand(1000, 4*len(bodies))
quats = quats / np.linalg.norm(quats, axis=0)
quats = quats.reshape(-1, len(bodies), 4)
quats.shape

(1000, 4, 4)

In [3]:
# Building test_oriantation
t = 0
dt = 1
with open("oriantation.sto", 'w') as f:
                    f.write("DataRate=200.000000\n")
                    f.write("DataType=Quaternion\n")
                    f.write("version=3\n")
                    f.write("OpenSimVersion=4.4-2022-07-23-0e9fedc\n")
                    f.write("endheader\n")
                    f.write('\t'.join(["time"] + bodies))
                    f.write('\n')
                    f.write(str(t))
                    for q in quats:
                        for num_bodies in range(len(bodies)):
                            f.write('\t')
                            f.write(f"{q[num_bodies][0]},{q[num_bodies][1]},{q[num_bodies][2]},{q[num_bodies][3]}")
                        
                        f.write("\n")
                        t += dt
                        f.write(str(t))
                    # for q in quats:
                    #     f.write('\t')
                    #     f.write(f"{q[-1][0]},{q[-1][1]},{q[-1][2]},{q[-1][3]}")
                    # f.write("\n")

with open("oriantation.sto", "r") as file:
    lines = file.readlines()

with open("oriantation.sto", "w") as file:
    file.writelines(lines[:-1])  # Write all lines except the last one


In [4]:
# Building oriantation_calib
t = 0
with open("oriantation_calib.sto", 'w') as f:
                    f.write("DataRate=200.000000\n")
                    f.write("DataType=Quaternion\n")
                    f.write("version=3\n")
                    f.write("OpenSimVersion=4.4-2022-07-23-0e9fedc\n")
                    f.write("endheader\n")
                    f.write('\t'.join(["time"] + bodies))
                    f.write('\n')
                    f.write(str(t))
                    for q in quats:
                        for num_bodies in range(len(bodies)):
                            f.write('\t')
                            f.write("1,0,0,0")
                        break

In [5]:
# ORIGINAL
imuPlacer = osim.IMUPlacer()
imuPlacer.set_model_file('Rajagopal_2015.osim')
imuPlacer.set_orientation_file_for_calibration("oriantation_calib.sto")
imuPlacer.set_sensor_to_opensim_rotations(osim.Vec3(0, 0, 0))
imuPlacer.set_base_imu_label('pelvis_imu')
imuPlacer.set_base_heading_axis('z')
imuPlacer.run(False)
model = imuPlacer.getCalibratedModel()
model.printToXML('calibrated_Rajagopal_2015.osim')
model = osim.Model('calibrated_Rajagopal_2015.osim')

quatTable = osim.TimeSeriesTableQuaternion("oriantation.sto")
orientationsData = osim.OpenSenseUtilities.convertQuaternionsToRotations(quatTable)
oRefs = osim.OrientationsReference(orientationsData)
mRefs = osim.MarkersReference()
coordinateReferences = osim.SimTKArrayCoordinateReference()
# model.setUseVisualizer(True)
# state = model.initSystem()

# ikSolver = osim.InverseKinematicsSolver(model, mRefs, oRefs, coordinateReferences, 10)
# ikSolver.setAccuracy = 1e-3
# state.setTime(0.)
# ikSolver.assemble(state)


### GPT

In [6]:
import opensim as osim

# === Calibration using IMUPlacer ===
imuPlacer = osim.IMUPlacer()
imuPlacer.set_model_file('Rajagopal_2015.osim')
imuPlacer.set_orientation_file_for_calibration("oriantation_calib.sto")
imuPlacer.set_sensor_to_opensim_rotations(osim.Vec3(0, 0, 0))
imuPlacer.set_base_imu_label('pelvis_imu')
imuPlacer.set_base_heading_axis('z')
imuPlacer.run(False)
model = imuPlacer.getCalibratedModel()
model.printToXML('calibrated_Rajagopal_2015.osim')
model = osim.Model('calibrated_Rajagopal_2015.osim')

# === Create OrientationsReference from oriantation.sto ===
quatTable = osim.TimeSeriesTableQuaternion("oriantation.sto")
orientationsData = osim.OpenSenseUtilities.convertQuaternionsToRotations(quatTable)
oRefs = osim.OrientationsReference(orientationsData)
mRefs = osim.MarkersReference()
coordinateReferences = osim.SimTKArrayCoordinateReference()

# Enable the visualizer so the model is shown in 3D.
model.setUseVisualizer(True)
state = model.initSystem()

# === Set up the Inverse Kinematics Solver ===
ikSolver = osim.InverseKinematicsSolver(model, mRefs, oRefs, coordinateReferences, 10)
ikSolver.setAccuracy(1e-3)

# Get the time vector from the orientation table.
times = quatTable.getIndependentColumn()
numFrames = len(times)
print("Animating %d frames." % numFrames)

# === Animate the Model ===
# Loop through each time step in the orientation data, update state, run IK, and update the visualizer.
for i in range(numFrames):
    t = times[i]
    state.setTime(t)
    ikSolver.assemble(state)
    # Update the visualizer to show the current state.
    model.getVisualizer().show(state)


Animating 1000 frames.


RuntimeError: std::exception in 'void OpenSim::ModelVisualizer::show(SimTK::State const &) const': SimTK Exception thrown at VisualizerProtocol.cpp:603:
  Error detected by Simbody method VisualizerProtocol: An attempt to write() 4 bytes to pipe 5 failed with errno=22 (Invalid argument).
  (Required condition 'status!=-1' was not met.)


## Using real quat values

In [9]:
# Importing data
df = pd.read_csv("Data/Hojjati_Normal.csv")
# 2, 8, 9, 10, 11, 12 I want these IDs
df = df[df["ID"].isin([2, 8, 9, 10, 11, 12])]

In [10]:
#TODO: After fixing the sensors' data collection this won't be necassary
# Replacing the Quat and LAcc values
df = df.copy()
df[['Quaternion w', 'Quaternion x', 'Quaternion y']] = df[['Linear Acceleration x','Linear Acceleration y', 'Linear Acceleration z']].values
df[['Linear Acceleration x','Linear Acceleration y', 'Linear Acceleration z']] = df[['Quaternion w', 'Quaternion x', 'Quaternion y']].values

### user

IMU to Bpdy part

`Choseb from following body parts:`


hand_r_imu, hand_l_imu, radius_r_imu, radius_l_imu, humerus_r_imu, humerus_l_imu

torso_imu, pelvis_imu, 

femur_r_imu, femur_l_imu, tibia_r_imu, tibia_l_imu, calcn_r_imu, calcn_l_imu,
head

In [11]:
# Assigning each IMU to a body part 
id_to_body = {
            2: 'pelvis_imu',
            8: 'hand_r_imu',
            9: 'torso_imu',
            10: 'tibia_l_imu'
            # 3: 'l_leg',
             }

# Create a new column for body part
df['body_part'] = df['ID'].map(id_to_body)

df.dropna(inplace=True) # Droping the values that have no IDs

Selecting Joint angles to save

In [12]:
# Joint
joint_name = ['pelvis_tilt',
 'pelvis_list',
 'pelvis_rotation',
#  'pelvis_tx',
#  'pelvis_ty',
#  'pelvis_tz',
 'hip_flexion_r',
 'hip_adduction_r',
 'hip_rotation_r',
#  'knee_angle_r',
#  'knee_angle_r_beta',
#  'ankle_angle_r',
#  'subtalar_angle_r',
#  'mtp_angle_r',
#  'hip_flexion_l',
#  'hip_adduction_l',
#  'hip_rotation_l',
#  'knee_angle_l',
#  'knee_angle_l_beta',
#  'ankle_angle_l',
#  'subtalar_angle_l',
#  'mtp_angle_l',
#  'lumbar_extension',
#  'lumbar_bending',
#  'lumbar_rotation',
 'arm_flex_r',
 'arm_add_r',
 'arm_rot_r',
#  'elbow_flex_r',
#  'pro_sup_r',
#  'wrist_flex_r',
#  'wrist_dev_r',
#  'arm_flex_l',
#  'arm_add_l',
#  'arm_rot_l',
#  'elbow_flex_l',
#  'pro_sup_l',
#  'wrist_flex_l',
#  'wrist_dev_l'
 ]

### Giving data to OpenSim

In [13]:
body_quaternions = {}
for _, row in df.iterrows():
    part = id_to_body[row['ID']]
    quat = (row['Quaternion w'], row['Quaternion x'], row['Quaternion y'], row['Quaternion z'])
    if part not in body_quaternions:
        body_quaternions[part] = []
    body_quaternions[part].append(quat) # Each body part has its own quat values


In [14]:
# Building oriantation
t = 0
dt = 1

num_rows = len(next(iter(body_quaternions.values())))

with open("oriantation.sto", 'w') as f:
    f.write("DataRate=200.000000\n")
    f.write("DataType=Quaternion\n")
    f.write("version=3\n")
    f.write("OpenSimVersion=4.4-2022-07-23-0e9fedc\n")
    f.write("endheader\n")
    
    # Write header: "time" followed by each body part name
    header = '\t'.join(["time"] + list(body_quaternions.keys()))
    f.write(header + "\n")
    
    # Write each row of data
    for i in range(num_rows):
        # Start each line with the current time value
        row = [str(t)]
        
        # For each body part, get the i-th quaternion and format it as a comma-separated string
        for body in body_quaternions.keys():
            q = body_quaternions[body][i]
            q_str = f"{q[0]},{q[1]},{q[2]},{q[3]}"
            row.append(q_str)
        
        # Join the row values with a tab and write to file
        f.write('\t'.join(row) + "\n")
        t += dt


In [17]:
# Calibration
from scipy.spatial.transform import Rotation
import numpy as np

from orientation import get_body_quaternions
from data import df, csv_to_df

cal_df = csv_to_df("Data/Hojjati_Stationary_Calibration_1.csv")
cal_quat = get_body_quaternions(cal_df)

mean_cal_quat = {}
for key, value in cal_quat.items():
    mean = Rotation.from_quat(value).mean().as_quat() # Calculate the mean
    mean_cal_quat.setdefault(key, []).append(mean)    # Add to the dict

In [18]:
# Building oriantation_calib
t = 0
with open("oriantation_calib.sto", 'w') as f:
                    f.write("DataRate=200.000000\n")
                    f.write("DataType=Quaternion\n")
                    f.write("version=3\n")
                    f.write("OpenSimVersion=4.4-2022-07-23-0e9fedc\n")
                    f.write("endheader\n")
                    f.write('\t'.join(["time"] + list(mean_cal_quat.keys())))
                    f.write('\n')
                    f.write(str(t))
                    for key in mean_cal_quat:
                        f.write('\t')
                        f.write(f"{mean_cal_quat[key][0][0]}, {mean_cal_quat[key][0][1]}, {mean_cal_quat[key][0][2]}, {mean_cal_quat[key][0][3]}")

In [19]:
# Building oriantation_calib
t = 0
with open("oriantation_calib.sto", 'w') as f:
                    f.write("DataRate=200.000000\n")
                    f.write("DataType=Quaternion\n")
                    f.write("version=3\n")
                    f.write("OpenSimVersion=4.4-2022-07-23-0e9fedc\n")
                    f.write("endheader\n")
                    f.write('\t'.join(["time"] + list(body_quaternions.keys())))
                    f.write('\n')
                    f.write(str(t))
                    for num_bodies in range(len(body_quaternions)):
                        f.write('\t')
                        f.write("1,0,0,0")

In [1]:
# === Calibration using IMUPlacer ===
imuPlacer = osim.IMUPlacer()
imuPlacer.set_model_file('Rajagopal_2015.osim')
imuPlacer.set_orientation_file_for_calibration("oriantation_calib.sto")
imuPlacer.set_sensor_to_opensim_rotations(osim.Vec3(0, 0, 0))
imuPlacer.set_base_imu_label('pelvis_imu')
imuPlacer.set_base_heading_axis('z')
imuPlacer.run(False)
model = imuPlacer.getCalibratedModel()
model.printToXML('calibrated_Rajagopal_2015.osim')
model = osim.Model('calibrated_Rajagopal_2015.osim')

# === Create OrientationsReference from oriantation.sto ===
quatTable = osim.TimeSeriesTableQuaternion("oriantation.sto")
orientationsData = osim.OpenSenseUtilities.convertQuaternionsToRotations(quatTable)
oRefs = osim.OrientationsReference(orientationsData)
mRefs = osim.MarkersReference()
coordinateReferences = osim.SimTKArrayCoordinateReference()

# Enable the visualizer so the model is shown in 3D.
model.setUseVisualizer(True)
state = model.initSystem()

# === Set up the Inverse Kinematics Solver ===
ikSolver = osim.InverseKinematicsSolver(model, mRefs, oRefs, coordinateReferences, 10)
ikSolver.setAccuracy(1e-3)

# Get the time vector from the orientation table.
times = quatTable.getIndependentColumn()
numFrames = len(times) // 5
print("Animating %d frames." % numFrames)

# === Animate the Model ===
# Loop through each time step in the orientation data, update state, run IK, and update the visualizer.
joint_angle = {}

try:
    for i in range(numFrames):
        t = times[i*5]
        state.setTime(t)
        ikSolver.assemble(state)
        # Update the visualizer to show the current state.
        model.getVisualizer().show(state)
        model.getVisualizer().getSimbodyVisualizer().setShowSimTime(True)
    #########################################################################################
        # Get the JointSet from the model.
        jointSet = model.getJointSet()

        # Loop through every joint in the set.
        for i in range(jointSet.getSize()):
            joint = jointSet.get(i)
            
            # Loop through all coordinates for this joint.
            for j in range(joint.numCoordinates()):
                coord = joint.get_coordinates(j)
                if coord.getName() in joint_name:
                    angle = coord.getValue(state)
                    # print("   Coordinate:", coord.getName(), "Angle:", angle)
                    joint_angle.setdefault(coord.getName(), []).append(angle) ## coordination name: angle ## 
except Exception as e:
    print(f"Error at frame {i}: {e}") 


now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output = pd.DataFrame(joint_angle)
output.to_csv(f"Outputs/Joint_Angles_{now}.csv")


NameError: name 'osim' is not defined